In [1]:
from glob import glob
import os

import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

import threading

from src.utils.normalize_text import DocumentFormatter
from src.anexo_b.process_b_attachment import anexo_b

In [2]:
dir_Arquivos = Path(r"data\Input\Resumos")
output_pdf = Path(r"data\Output\Resumos\pdf")
output_excel = Path(r"data\Output\Resumos\excel")

In [3]:
itens = pd.read_csv(
    Path.joinpath(dir_Arquivos, "Contratos.csv"), dtype=str, names=["Contrato"]
)
comparativo = pd.read_excel(
    Path.joinpath(dir_Arquivos, "Status_2014.xlsx"), dtype={"Contrato": str}
)
caminho_base = Path.joinpath(dir_Arquivos, "modelo_piscofins.xlsx")
path_base = Path.joinpath(dir_Arquivos, "DOC_BASE_Reprocessado.xlsx")


In [4]:
df_quadro_um = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_um.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_dois = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_dois.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_tres = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_tres.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_quatro = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_quatro.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_cinco = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_cinco.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)
df_quadro_seis = pd.read_csv(
    Path.joinpath(dir_Arquivos, r"quadros\quadro_seis.csv"),
    encoding="latin1",
    sep=",",
    dtype=str,
)

In [5]:
lst_itens = itens["Contrato"].tolist()

comparativo["Contrato"] = comparativo["Contrato"].astype(str)
comparativo = comparativo[comparativo["Contrato"].isin(lst_itens)]

## Processamento

In [6]:
tipos_colunas = {
    "nº do contrato": str,
    "Razão Social_Nome Completo do Arrendatário": str,
    "CNPJ_CPF do Arrendatário": str,
    "Nº de Parcelas Contratadas": str,
    "Valor do Bem": float,
}
data_columns = ["data do contrato", "Liquidação"]

df_base = pd.read_excel(path_base, dtype=tipos_colunas)
df_base["N° de parcelas contratadas"] = (
    pd.to_numeric(df_base["N° de parcelas contratadas"], errors="coerce")
    .fillna(0)
    .astype(int)
)

df_base["nº do contrato"] = df_base["nº do contrato"].str.zfill(7)

df_base["CNPJ_CPF do Arrendatário"] = df_base["CNPJ_CPF do Arrendatário"].apply(
    DocumentFormatter.format_documents
)

df_base["Razão Social_Nome Completo do Arrendatário"] = df_base[
    "Razão Social_Nome Completo do Arrendatário"
].apply(DocumentFormatter.to_pascal_case)
df_base = DocumentFormatter.format_date_columns(df_base, data_columns)
df_base["nº do contrato"] = df_base["nº do contrato"].astype(str)

In [7]:
df_base = df_base[
    [
        "nº do contrato",
        "Razão Social_Nome Completo do Arrendatário",
        "CNPJ_CPF do Arrendatário",
        "Valor do Bem",
        "data do contrato",
        "Liquidação",
        "N° de parcelas contratadas",
    ]
]
df_base.columns = [
    "Nº do Contrato",
    "Cliente",
    "CNPJ/CPF",
    "Valor do Bem",
    "Data do Contrato",
    "Data de Liquidação",
    "Quantidade de Parcelas",
]

In [8]:
contratos_validos = comparativo["Contrato"].values.tolist()

In [9]:
df_base["Nº do Contrato"].unique()

array(['0141715', '0158876', '0337640', ..., '5221444', '5221523',
       '5221862'], shape=(458118,), dtype=object)

In [10]:
df_base = df_base[df_base["Nº do Contrato"].isin(contratos_validos)]
info_contratos = df_base.groupby("Nº do Contrato")
replace = True

In [11]:
# warnings.filterwarnings("ignore")
thread_limit = 30
idx = 0
threads = []

pbar = tqdm(info_contratos, total=len(info_contratos))
for contrato, info in pbar:
    pbar.set_description(f"Processando contrato: {contrato}")
    thread = threading.Thread(
        target=anexo_b().criar_folha_rosto,
        args=(
            contrato,
            info,
            comparativo,
            output_excel,
            df_quadro_um,
            df_quadro_dois,
            df_quadro_tres,
            df_quadro_quatro,
            df_quadro_cinco,
            df_quadro_seis,
            caminho_base,
            replace,
        ),
    )
    thread.start()
    threads.append(thread)

    if len(threads) >= thread_limit or idx == len(info_contratos) - 1:
        # Aguarde até que todas as threads terminem
        for thread in threads:
            thread.join()
        threads = []
    idx += 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:

'''files = glob(os.path.join(output_excel, "*.xlsx"))
output_pdf = os.path.abspath(output_pdf)

pbar = tqdm(files, total=len(files))
for file in pbar:
    pbar.set_description(f"Convertendo {os.path.basename(file)} para PDF")
    anexo_b.xlsx_to_pdf_one_page(os.path.abspath(file), output_pdf)'''

'files = glob(os.path.join(output_excel, "*.xlsx"))\noutput_pdf = os.path.abspath(output_pdf)\n\npbar = tqdm(files, total=len(files))\nfor file in pbar:\n    pbar.set_description(f"Convertendo {os.path.basename(file)} para PDF")\n    anexo_b.xlsx_to_pdf_one_page(os.path.abspath(file), output_pdf)'

In [13]:
'''files = glob(os.path.join(output_excel, "*.xlsx"))
output_pdf = Path(output_pdf).resolve()

pbar = tqdm(files, total=len(files))

for file in pbar:
    nome_base = Path(file).stem
    pdf_destino = output_pdf / f"{nome_base}.pdf"

    if pdf_destino.exists():
        pbar.set_description(f"Pulando (já existe): {nome_base}.pdf")
        continue

    pbar.set_description(f"Convertendo {nome_base} para PDF")
    anexo_b.xlsx_to_pdf_one_page(os.path.abspath(file), str(output_pdf))'''

'files = glob(os.path.join(output_excel, "*.xlsx"))\noutput_pdf = Path(output_pdf).resolve()\n\npbar = tqdm(files, total=len(files))\n\nfor file in pbar:\n    nome_base = Path(file).stem\n    pdf_destino = output_pdf / f"{nome_base}.pdf"\n\n    if pdf_destino.exists():\n        pbar.set_description(f"Pulando (já existe): {nome_base}.pdf")\n        continue\n\n    pbar.set_description(f"Convertendo {nome_base} para PDF")\n    anexo_b.xlsx_to_pdf_one_page(os.path.abspath(file), str(output_pdf))'

In [14]:
#anexo_b().process_anexo_pdfs(Path(output_pdf))